# Lecture 1 – Introduction

## Data 94, Spring 2021

This is a Jupyter notebook. We'll write all of our code in this class in a Jupyter notebook.

Today, don't worry about how any of this works. Throughout the semester, we'll learn how each of these pieces work.

**Note: The maps in this notebook will not load correctly in Safari if you're on a Mac; use Chrome.**

In [ ]:
!pip install googlemaps

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import googlemaps

# "KEY" is sensitive (it's almost like a password).
# I've removed mine in this posted version of the notebook, so some of the code won't run for you.
KEY = 'xxx'

## California universities

Here, we'll load in data about all public universities in California. The data comes from [this Wikipedia article](https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_California).

In [ ]:
uni = Table.read_table('data/california_universities.csv')

uni = uni.with_columns(
    'Enrollment', uni.apply(lambda s: int(s.replace(',', '')), 'Enrollment'),
    'Founded', uni.apply(lambda s: int(s.replace('*', '')), 'Founded')
)

Data is often stored in tables. In about a month, we'll become very, very familiar with how tables work. But for now, let's just observe.

In [ ]:
uni.show(15)

Let's start asking questions.

### What's the largest public university in California?

In [ ]:
uni.sort('Enrollment', descending = True)

In [ ]:
uni.sort('Enrollment', descending = True).barh('Name', 'Enrollment')

### What's the oldest public university in California? 🤔

In [ ]:
uni.sort('Founded')

In [ ]:
uni_copy = uni.sort('Founded').with_columns('Total Universities', np.arange(1, uni.num_rows + 1))
uni_copy.plot('Founded', 'Total Universities')

Let's add some spice.

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = uni_copy.column('Founded'), 
               y = uni_copy.column('Total Universities'), 
               hovertext = uni_copy.column('Name'),
               mode = 'markers',
              )
)

fig.add_trace(
    go.Scatter(x = uni_copy.column('Founded'), 
               y = uni_copy.column('Total Universities'),
               line = dict(color = 'blue'),
              )
)

fig.update_layout(title = 'Total Number of Public Universities in California by Year',
                  xaxis_title = 'Year',
                  yaxis_title = 'Total Universities')

fig.show()

### Where are the public universities in California located?

In [ ]:
uni

What if we want to plot these on a map?

It turns out that we can use Google Maps to do it – using code! (Edit: this won't run for you.)

In [ ]:
gmaps = googlemaps.Client(key = KEY)

Let's try a location that we all know and love... (Edit: this also won't run for you.)

In [ ]:
geocode_result = gmaps.geocode('taco bell southside berkeley')
geocode_result

In [ ]:
def school_to_lat_lon(school):
    res = gmaps.geocode(school)[0]
    lat = res['geometry']['location']['lat']
    lng = res['geometry']['location']['lng']
    return lat, lng

Given the latitude and longitude of a location, we can plot it on a map!

In [ ]:
# lat_lon = uni.apply(lambda name: school_to_lat_lon(name), 'Name')

In [ ]:
# uni_locations = uni.with_columns(
# 'Latitude', lat_lon[:, 0],
# 'Longitude', lat_lon[:, 1]).select('Latitude', 'Longitude', 'Name').relabeled('Name', 'labels')

In [ ]:
uni_locations = Table.read_table('data/uni_locations.csv')

In [ ]:
Marker.map_table(uni_locations, 
                 marker_icon='info-sign')

It would be nice if this were color-coded based on UC vs. CSU. We can do that!

In [ ]:
uni_locations_separate = uni_locations.with_columns(
    'colors', uni_locations.apply(lambda s: 'blue' if 'State' not in s else 'red', 'labels')
)

In [ ]:
Marker.map_table(uni_locations_separate, marker_icon='info-sign')